In [53]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_binary
import pandas as pd
import pprint
from selenium.webdriver.common.keys import Keys


In [345]:
years = ['2018','2017','2016','2015','2014','2012','2011','2010','2009','2008','2007','2006','2005','2004','2003','2002','2001','1999']
berlin_url = 'https://www.bmw-berlin-marathon.com/en/impressions/statistics-and-history/results-archive/'
driver = webdriver.Chrome()
for year in years:
    driver.get(berlin_url)
    time.sleep(1)
    #accept cookies
    driver.find_element_by_xpath('//button[@data-cookiefirst-action="accept"]').click()
    time.sleep(1)
    #get specific year
    driver.find_element_by_xpath('//select[@class="events border-input"]').click()
    driver.find_element_by_xpath(f"//option[contains(text(),{year})]").click()
    master_results = []
    #get number of pages
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pages = soup.find_all('a',{"class": "page-link"})
    pages_text = [res.getText(',') for res in pages]
    #open all the splits
    for i in range(int(pages_text[7])-1):
        time.sleep(.7)
        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.PAGE_UP)
        time.sleep(.7)
        btns = driver.find_elements(By.XPATH, f'//td[@class= " details-control"]')
        for button in btns:
            button.click()
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        #scrape current page
        results = soup.find_all('tr')
        results_text = [res.getText(',') for res in results]
        accum = 0
        for i in range(21):
            master_results.append([results_text[accum] + ',' + results_text[accum+1]])
            accum += 2

        time.sleep(.2)

        #next page
        driver.find_element_by_xpath("//*[contains(text(), 'Next')]").click()

    #format data
    master_results_flat = [x for x in master_results]
    for i in range(len(master_results_flat)):
        master_results_flat[i] = master_results_flat[i][0].split(',')

    for i in range(20):
        for i in master_results_flat:
            if len(i) == 12:
                master_results_flat.remove(i)
            if len(i) == 28:
                del i[5]
            elif len(i) != 27:
                master_results_flat.remove(i)

    #save to dataframe and csv
    df = pd.DataFrame(master_results_flat, columns = ['Overall_Place', 'First_Name','Last_Name', 'Country','SN', 'Sex','Net_time','Gross_time','split_times', '5k', '5k_split','10k', '10k_split','15k', '15k_split','20k', '20k_split','HM', 'HM_split','25k', '25k_split','30k', '30k_split','35k', '35k_split','40k', '40k_split'])
    
    df.to_csv(f'data/berlin_{year}.csv', header=True, index=False)
#print(master_results)


time.sleep(5)
driver.close()
driver.quit()

ElementClickInterceptedException: Message: element click intercepted: Element <td>...</td> is not clickable at point (670, 28). Other element would receive the click: <a href="" class="nav-link" role="button" data-toggle="dropdown" aria-haspopup="true" aria-expanded="false">...</a>
  (Session info: chrome=101.0.4951.64)


In [324]:
int(pages_text[7])-1

1154

In [335]:
master_results_flat = [x for x in master_results]
for i in range(len(master_results_flat)):
    master_results_flat[i] = master_results_flat[i][0].split(',')
master_results_flat

[['Place',
  'First Name',
  'Last Name',
  'Nationality',
  '#SN',
  'Club',
  'Sex',
  'Net',
  'Gross',
  'Photo',
  'Certificate',
  ''],
 ['1',
  'Kenenisa',
  'Bekele',
  'ETH',
  '2',
  'Äthiopien',
  'M',
  '02:01:41',
  '02:01:41',
  'Split times',
  '5km',
  '00:14:24',
  '10km',
  '00:28:53',
  '15km',
  '00:43:29',
  '20km',
  '00:57:58',
  'HM',
  '01:01:05',
  '25km',
  '01:12:30',
  '30km',
  '01:26:55',
  '35km',
  '01:41:15',
  '40km',
  '01:55:30'],
 ['2',
  'Birhanu',
  'Legese',
  'ETH',
  '5',
  'Äthiopien',
  'M',
  '02:02:48',
  '02:02:48',
  'Split times',
  '5km',
  '00:14:25',
  '10km',
  '00:28:53',
  '15km',
  '00:43:29',
  '20km',
  '00:57:58',
  'HM',
  '01:01:05',
  '25km',
  '01:12:30',
  '30km',
  '01:26:53',
  '35km',
  '01:41:02',
  '40km',
  '01:56:00'],
 ['3',
  'Sisay',
  'Lemma',
  'ETH',
  '4',
  'Äthiopien',
  'M',
  '02:03:36',
  '02:03:36',
  'Split times',
  '5km',
  '00:14:25',
  '10km',
  '00:28:52',
  '15km',
  '00:43:29',
  '20km',
  '00:

In [341]:
for i in master_results_flat:
    if len(i) == 12:
        master_results_flat.remove(i)
    if len(i) == 28:
        del i[5]
    elif len(i) != 27:
        print(i)
        master_results_flat.remove(i)
master_results_flat[1]

['2',
 'Birhanu',
 'Legese',
 'ETH',
 '5',
 'M',
 '02:02:48',
 '02:02:48',
 'Split times',
 '5km',
 '00:14:25',
 '10km',
 '00:28:53',
 '15km',
 '00:43:29',
 '20km',
 '00:57:58',
 'HM',
 '01:01:05',
 '25km',
 '01:12:30',
 '30km',
 '01:26:53',
 '35km',
 '01:41:02',
 '40km',
 '01:56:00']

In [342]:
df = pd.DataFrame(master_results_flat, columns = ['Overall_Place', 'First_Name','Last_Name', 'Country','SN', 'Sex','Net_time','Gross_time','split_times', '5k', '5k_split','10k', '10k_split','15k', '15k_split','20k', '20k_split','HM', 'HM_split','25k', '25k_split','30k', '30k_split','35k', '35k_split','40k', '40k_split'])

In [343]:
df.tail()

,Overall_Place,First_Name,Last_Name,Country,SN,Sex,Net_time,Gross_time,split_times,5k,...,HM,HM_split,25k,25k_split,30k,30k_split,35k,35k_split,40k,40k_split
43700,30730,Joseph,Emas,CAN,18404,M,06:57:29,07:12:47,Split times,5km,...,HM,03:03:52,25km,03:45:18,30km,04:42:41,35km,05:42:30,40km,06:35:21
43701,30731,Hans-Jürgen,Stampe,GER,3745,M,06:57:57,07:53:00,Split times,5km,...,HM,03:14:30,25km,03:55:22,30km,04:48:46,35km,05:41:57,40km,06:35:57
43702,30732,Dietmar,Schultejans,GER,36417,M,06:58:40,06:59:58,Split times,5km,...,HM,03:12:59,25km,03:51:51,30km,04:43:18,35km,05:38:42,40km,06:36:32
43703,30733,Bruno,Cavallini,ITA,15844,M,06:58:43,07:35:17,Split times,5km,...,HM,02:40:08,25km,03:24:20,30km,04:32:14,35km,05:33:00,40km,06:33:43
43704,13238,Maki,Murao,JPN,63886,W,07:02:03,07:13:25,Split times,5km,...,HM,02:57:39,25km,03:47:17,30km,04:46:03,35km,05:42:29,40km,06:39:10


In [344]:
df.to_csv('data/berlin_2019.csv', header=True, index=False)